In [1]:
import pandas as pd
import torch
from numpy import isnan
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from torch import nn


In [2]:
ibm_2014 = pd.read_csv('data/raw/2014_7_18_ibm_CDEdata.csv',delimiter=',',encoding='latin_1')
ibm_2014.head()

,Topic,Article,Claim,CDE,Type 1,Type 2
0,the sale of violent video games to minors,Video game controversies,exposure to violent video games causes at leas...,The most recent large scale meta-anlysis-- exa...,STUDY,NaN
1,the sale of violent video games to minors,Video game controversies,exposure to violent video games causes at leas...,One study did find an increase in reports of b...,STUDY,NaN
2,the sale of violent video games to minors,Video game controversies,exposure to violent video games causes at leas...,when one combines all relevant empirical studi...,STUDY,NaN
3,the sale of violent video games to minors,Video game controversies,exposure to violent video games causes at leas...,Research published in 2001 suggested that viol...,STUDY,NaN
4,the sale of violent video games to minors,Video game controversies,exposure to violent video games causes at leas...,In 2004 the American Psychological Association...,STUDY,EXPERT


In [4]:
ibm_2015 = pd.read_csv('data/raw/IBM_Debater_2015_evidence.csv',delimiter='\t',encoding='latin_1')
ibm_2015.head()

,Topic,Claim,CDE,Type 1
0,This house believes that the sale of violent v...,Exposure to violent video games causes at leas...,A 2001 study found that exposure to violent vi...,[STUDY]
1,This house believes that the sale of violent v...,Exposure to violent video games causes at leas...,The most recent large scale meta-anlysis-- exa...,[STUDY]
2,This house believes that the sale of violent v...,Exposure to violent video games causes at leas...,One study did find an increase in reports of b...,[STUDY]
3,This house believes that the sale of violent v...,Exposure to violent video games causes at leas...,when one combines all relevant empirical studi...,[STUDY]
4,This house believes that the sale of violent v...,Exposure to violent video games causes at leas...,Research published in 2001 suggested that viol...,[STUDY]


In [5]:
def str_to_list(labels:str) -> list:
    tags = labels.translate(str.maketrans({'[' : '', ']': '',' ' : ''}))
    tags = tags.split(',')
    return tags

In [9]:
sent = {}

for index, row in ibm_2014.iterrows():
    try:
        sent[row['CDE']].append(row['Type 1'])
        if type(row['Type 2']) is not float:
            sent[row['CDE']].append(row['Type 2']) 
    except KeyError:
        sent[row['CDE']] = [row['Type 1']]
        if type(row['Type 2']) is not float:
            sent[row['CDE']].append(row['Type 2']) 
    except Exception as E:
        print(f'{index} : {E}')

        
for index, row in ibm_2015.iterrows():
    labels = str_to_list(row['Type 1'])
    try:
        sent[row['CDE']]+=labels
    except KeyError:
        sent[row['CDE']] = labels
    except Exception as E:
        print(f'{index} : {E}')

In [13]:
for key, val in sent.items():
    sent[key] = list(set(sent[key]))

In [40]:
def retrieve_new_label(labels:list) -> str:
    if 'ANECDOTAL' in labels:
        return 'low'
    elif 'STUDY' in labels:
        return 'high'
    elif 'EXPERT' in labels:
        return 'medium'
        
        

In [41]:
new_format_data = {}
for key, val in sent.items():
    new_format_data[key] = retrieve_new_label(val)


In [44]:
df = pd.DataFrame.from_dict({'text':list(new_format_data.keys()),'label':list(new_format_data.values())})

In [46]:
df.head()

,text,label
0,The most recent large scale meta-anlysis-- exa...,high
1,One study did find an increase in reports of b...,high
2,when one combines all relevant empirical studi...,high
3,Research published in 2001 suggested that viol...,high
4,In 2004 the American Psychological Association...,high


In [42]:
from collections import Counter
Counter(list(new_format_data.values()))

Counter({'medium': 1491, 'high': 785, 'low': 531})

In [47]:
df.to_csv('credibility.csv',index=False)